In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report as cr
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

E:\anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


### Load dataset and split them into train and test set

In [2]:
filepath = 'C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/'
#####################   Modify above path   #########################
os.chdir(filepath)

# testpath=filepath+'/data/test_set/'
trainpath=filepath+'/data/train_set/'

# test_image_dir = testpath + "images/"
# test_pt_dir = testpath + "points/"
train_image_dir = trainpath + "images/"
train_pt_dir = trainpath + "points/"

import sklearn.metrics.pairwise
def pairwise_dist(vec):
    dist  = sklearn.metrics.pairwise_distances(vec, metric='euclidean')
    np.fill_diagonal(dist, np.nan)
    return dist
def feature(fiducial_pt_list,index):
    pairwise_dist_feature = pairwise_dist(fiducial_pt_list[index]).flatten()
    pairwise_dist_feature = pairwise_dist_feature[~np.isnan(pairwise_dist_feature)]
    return pairwise_dist_feature

f0 = time.time()
dataDir = train_pt_dir
fiducial_pt_list = []
filelist = []
for file in os.listdir(dataDir):
    filelist.append(file)
filelist.sort()
for file in filelist:
    fiducial_pt_list.append(scipy.io.loadmat(dataDir+file))
    l = []
for i in range(len(fiducial_pt_list)):
    if 'faceCoordinatesUnwarped' in fiducial_pt_list[i].keys():
        l.append(fiducial_pt_list[i]['faceCoordinatesUnwarped'])
    else:
        l.append(fiducial_pt_list[i]['faceCoordinates2'])
        
fiducial_pt_list = l


X = pd.DataFrame(np.zeros((2500, 6006)))
for i in range(2500):
    X.iloc[i,:] = np.round(feature(fiducial_pt_list, i).flatten(), 0)
y =pd.read_csv(trainpath+'label.csv')['emotion_idx']
f1 = time.time()-f0

scaler = StandardScaler()
X = scaler.fit_transform(X)

train_x_dis,test_x_dis,train_y_dis,test_y_dis=train_test_split(X,y,test_size=0.2,random_state=3662)
X_dis=X
y_dis=y

### Use GridsearchCV to determine Parameters

In [3]:
# from sklearn.model_selection import GridSearchCV
# logistic_parm_gs = {
#     'C': [0.01,0.1,1,50] ,
#     'solver': ['newton-cg','lbfgs', 'sag','saga'],
#     'multi_class' : [ 'multinomial'],
#     'max_iter':[300],
#     'intercept_scaling':[1],
#     'dual':[False],
#     'fit_intercept':[True], 
#     'penalty':['l2'],
#     'tol':[0.0001],
# }
# logistic_model_gs = LogisticRegression()
# logistic_gsearch = GridSearchCV(estimator = logistic_model_gs, 
#                         param_grid = logistic_parm_gs, 
#                         scoring ='accuracy',
#                         cv = 3,
#                         n_jobs = 8,
#                         verbose=3,
#                         return_train_score=True
#                        )
# logistic_gsearch.fit(train_x_dis,train_y_dis)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:  9.1min
[Parallel(n_jobs=8)]: Done  48 out of  48 | elapsed: 29.2min finished
E:\anaconda\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=8,
             param_grid={'C': [0.01, 0.1, 1, 50], 'dual': [False],
                         'fit_intercept': [True], 'intercept_scaling': [1],
                         'max_iter': [300], 'multi_class': ['multinomial'],
                         'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
         

In [4]:
# from sklearn.metrics import accuracy_score 
# print("Training dataset Accuracy")
# train_preds = logistic_gsearch.predict(train_x_dis)
# train_accuracy = accuracy_score(train_y_dis, train_preds) 
# print(train_accuracy)

Training dataset Accuracy
0.9315


In [5]:
# print("Testing dataset Accuracy")
# test_preds = logistic_gsearch.predict(test_x_dis)
# test_accuracy = accuracy_score(test_y_dis, test_preds) 
# print(test_accuracy)

Testing dataset Accuracy
0.518


In [6]:
logistic_gsearch.best_params_

{'C': 0.01,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 300,
 'multi_class': 'multinomial',
 'penalty': 'l2',
 'solver': 'newton-cg',
 'tol': 0.0001}

### Use K-Fold to estimate the accuracy (54.00%)

In [11]:
logistic_model_final =LogisticRegression(C=0.01,
                                         dual=False,
                                         fit_intercept=True,
                                         intercept_scaling=1,
                                         max_iter=300,
                                         multi_class='multinomial',
                                         penalty='l2',
                                         solver='newton-cg',
                                         tol=0.0001)

In [12]:
# kfold = KFold(n_splits=10, shuffle=True)
# results = cross_val_score(logistic_model_final, X_dis,y_dis, cv=kfold)
# print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 54.00% (3.48%)


### Fit the final model and save it

In [13]:
t0 = time.time()
logistic_model_final.fit(X_dis,y_dis)
print("done in %0.3fs" % (time.time() - t0))

done in 72.013s


In [14]:
# joblib.dump(logistic_model_final,'logistic_model_final.m')#please change the path

['logistic_model_final.m']

In [ ]:
#joblib.load('C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/hanbojiao_test/doc/logistic_model_final.m')